In [ ]:
import cv2
import numpy as np
 
 
def add_transparent_image(background, foreground, x_offset=None, y_offset=None):
    bg_h, bg_w, bg_channels = background.shape
    fg_h, fg_w, fg_channels = foreground.shape

    assert bg_channels == 3, f'background image should have exactly 3 channels (RGB). found:{bg_channels}'
    assert fg_channels == 4, f'foreground image should have exactly 4 channels (RGBA). found:{fg_channels}'

    # center by default
    if x_offset is None: x_offset = (bg_w - fg_w) // 2
    if y_offset is None: y_offset = (bg_h - fg_h) // 2

    w = min(fg_w, bg_w, fg_w + x_offset, bg_w - x_offset)
    h = min(fg_h, bg_h, fg_h + y_offset, bg_h - y_offset)

    if w < 1 or h < 1: return

    # clip foreground and background images to the overlapping regions
    bg_x = max(0, x_offset)
    bg_y = max(0, y_offset)
    fg_x = max(0, x_offset * -1)
    fg_y = max(0, y_offset * -1)
    foreground = foreground[fg_y:fg_y + h, fg_x:fg_x + w]
    background_subsection = background[bg_y:bg_y + h, bg_x:bg_x + w]

    # separate alpha and color channels from the foreground image
    foreground_colors = foreground[:, :, :3]
    alpha_channel = foreground[:, :, 3] / 255  # 0-255 => 0.0-1.0

    # construct an alpha_mask that matches the image shape
    alpha_mask = np.dstack((alpha_channel, alpha_channel, alpha_channel))

    # combine the background with the overlay image weighted by alpha
    composite = background_subsection * (1 - alpha_mask) + foreground_colors * alpha_mask

    # overwrite the section of the background image that has been updated
    background[bg_y:bg_y + h, bg_x:bg_x + w] = composite

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('test.MOV')
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

frames = []
nbFrames = 78
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    frames.append(frame)
    if len(frames) >= nbFrames:
      break
    # frame  = cv2.resize(frame, (0,0), fx=0.5, fy=0.5) 
    #     # Convert image to image gray
    # tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      
    # # Applying thresholding technique
    # _, alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY)
      
    # # Using cv2.split() to split channels 
    # # of coloured image
    # b, g, r = cv2.split(frame)
      
    # # Making list of Red, Green, Blue
    # # Channels and alpha
    # rgba = [b, g, r, alpha]
      
    # # Using cv2.merge() to merge rgba
    # # into a coloured/multi-channeled image
    # dst = cv2.merge(rgba, 4)

    # if fullframe is None:
    #   fullframe = dst
    # else:
    #    fullframe = cv2.addWeighted(fullframe, 1.0, dst, 0.25, 0)

    # # # Display the resulting frame
    cv2.imshow('Frame', frame)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break

# Calculate blended image
fullframe = frames[0]
for i in range(len(frames)):
    alpha_merge = 1.0/(i + 1)
    beta_merge = 1.0 - alpha_merge
    # Convert image to image gray
    tmp = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
      
    # Applying thresholding technique
    _, alpha = cv2.threshold(tmp, 60, 255, cv2.THRESH_BINARY)
      
    # Using cv2.split() to split channels 
    # of coloured image
    b, g, r = cv2.split(frames[i])
      
    # Making list of Red, Green, Blue
    # Channels and alpha
    rgba = [b, g, r, alpha]
      
    # Using cv2.merge() to merge rgba
    # into a coloured/multi-channeled image
    transparentFrame = cv2.merge(rgba, 4)
    cv2.imwrite('f' + str(i) + '.png', transparentFrame)
    if i == 0:
        fullframe = transparentFrame
    else:
      # else:
      #     print(fullframe)
      #     fullframe =  add_transparent_image(fullframe, transparentFrame, 0, 0)
      #print(alpha)
      fullframe = cv2.addWeighted( fullframe, 0.5, transparentFrame, 0.5, 0.0)
      #cv2.imshow('Frame', dst)

 
cv2.imwrite('final.png', fullframe)


# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()